In [1]:
import numpy as np
import matplotlib.image as mpimg
import scipy
import os
import igor
from igor import igorpy as igor
from igor import binarywave as bw
import matplotlib.pyplot as plt

In [5]:
#The following functions for flattening the data were translated from Matlab functions written by the illustrious Matt Poss

def mZero(waveIn):
    #sets the average array value to 0
    average = np.mean(np.mean(waveIn))
    waveOut = waveIn - average
    return waveOut

def mFlattenY(ar):
    # Flattens each Y-direction line of the array one by one
    # Determine the dimensions of the array
    arDims = ar.shape;
    nRows = arDims[0];
    nCols = arDims[1];

    # Create temp array to return
    flar = ar.copy();

    # Determine total average of array
    tAvg = np.mean(np.mean(ar));

    # For each row of the array
    for cc1 in range(nCols):

        # Compute the linear fit

        # Compute col average
        temp = ar[:,cc1];
        avg = np.mean(temp);

        # Create 1D vector of n points spaced at 1 point intervals
        xAxis = np.transpose(np.linspace(1,nRows,nRows));

        # Take column of data and put it in a vector for polyfit() command
        yAxis = ar[:,cc1];

        # Fit a line to the column of data
        # There may be a faster way to do this
        polyCoeff = np.polyfit(xAxis, yAxis, 1);

        # Compute col average offset
        cAvgOff = avg - tAvg;

        # Adjust the value of each cell

        for r1 in range(nRows):
            # Compute cell offset
            off = (polyCoeff[0]*r1 + polyCoeff[1]) - avg;

            # Adjust cell value
            flar[r1,cc1] = ar[r1,cc1] - off - cAvgOff;
            #flar(r1,cc1) = ar(r1,cc1) - off;
            # This second line is a pure, true, 1st order line by line flatten
            # The first one has some flattening in the direction perpendicular
            # to the line

        # return flattened array

    # Zero the average height of the image
    flar = mZero(flar);
    return flar

In [11]:
#These functions I built to ingest an IBW from the Cypher and output all the processed channels

class chan(): #this class defines a channel from the ibw file
    def _init_(self,waves,labels,scanSize,n):
        self.raw = waves[:,:,n] #stores raw data
        self.flat = waves[:,:,n] #initialize flattened data as the raw data (so unflattened channels still have this)
        self.channel = labels[n] #channel labels will be used to loop through certain conditions
        self.size = scanSize #the size of the scan
        
    def _flatten_(self): #calls the flattening functions built by Matt Poss (above)
        self.flat = mFlattenY(self.raw)
        
    def _save_(self,path,ext): #saves the flattened image
        BuildFigure(self)
        #mpimg.imsave(path+str(self.channel)[2:-1]+ext,self.flat)
        plt.savefig(path+self.channel+ext)
        plt.close()
        
def Load_IBW(path): #this function uses the igor packaged and the defined class to load a full ibw file into channels
    ibw = bw.load(path) #loads an ibw [igor binary wave] file

    val = ibw.values() #these are all the values from the igor binary wave file
    waves = dict(list(val)[1])['wData'] #wData is where the channel information is stored
    labels = dict(list(val)[1])['labels'][2][1:] #pulls the labels for the channels
    
    temp = [str(labels[n])[2:-1] for n in range(len(labels))]
    labels = temp
    
    notes = str(dict(list(val)[1])['note']).split('\\')
    scanSize = float(notes[0].split(':')[1])

    N = waves.shape[2] #how many channels were taken

    #which channels are to be viewed
    toView = ['Height', 'Amplitude', 'Deflection', 'Phase', 'Frequency', 'Dissipation', 'Lateral']

    #whcih channels sould be flattened
    toFlatten = ['Height', 'Phase', 'Frequency']

    Chans = [] #initialize the list of channel objects
    {Chans.append(chan()) for n in range(N) if any([toView[q] in labels[n] for q in range(len(toView))])}; #create channel objects for desired channels
    {Chans[n]._init_(waves,labels,scanSize,n) for n in range(len(Chans))}; #initialize channels
    {Chans[n]._flatten_() for n in range(len(Chans)) if any([toFlatten[q] in Chans[n].channel for q in range(len(toFlatten))])}; #flatten appropriate channels
    
    return Chans

def SaveChans(Chans,path):
    if not os.path.exists(path+'PDF\\'): os.mkdir(path+'PDF\\')
    if not os.path.exists(path+'PNG\\'): os.mkdir(path+'PNG\\')
    if not os.path.exists(path+'EPS\\'): os.mkdir(path+'EPS\\')
    {Chans[n]._save_(path+'PNG\\','.png') for n in range(len(Chans))};
    {Chans[n]._save_(path+'PDF\\','.pdf') for n in range(len(Chans))};
    {Chans[n]._save_(path+'EPS\\','.eps') for n in range(len(Chans))};
    
def BuildFigure(Chan): #builds the full figure to be saved
    scanSize= Chan.size
    bases = {'Height':'m',
         'Amplitude':'m',
         'Deflection':'m',
         'Phase':'deg',
         'Frequency':'Hz',
         'Dissipation':'V',
         'Lateral':'V'}
    
    #base = bases[Chan.channel]
    base = bases[[list(bases.keys())[i] for i in range(len(bases)) if list(bases.keys())[i] in Chan.channel][0]]
    
    if np.amax(Chan.flat)>=1e3: pfx = ['k', 1e-3]
    elif np.amax(Chan.flat)>=1e6: pfx = ['M', 1e-6]
    elif np.amax(Chan.flat)<=1e-6: pfx = ['n', 1e9]
    elif np.amax(Chan.flat)<=1e-3: pfx = [r'$\mu$', 1e6]    
    elif np.amax(Chan.flat)<=1: pfx = ['m',1e3]
    else: pfx = ['',1]
    
    sfx = ['u',1e6]
    if scanSize <1e-6: sfx = ['n',1e9]
        
    fig = plt.figure()    
    fig.clear()
    
    im = plt.imshow(Chan.flat*pfx[1], cmap = plt.get_cmap('gray'), extent = (0, scanSize*sfx[1], 0, scanSize*sfx[1]));
    im.axes.set_xlabel(sfx[0]+'m')
    im.axes.set_ylabel(sfx[0]+'m')
    cb = plt.colorbar();
    plt.title(Chan.channel);
    cb.ax.set_xlabel(pfx[0]+base);    

In [14]:
def Cypher_IBW(directory):

    if not os.path.exists(directory+'Processed\\'): os.mkdir(directory+'Processed\\')
    for filename in os.listdir(directory):
        if filename.endswith(".ibw"): 
            path = directory+filename
            Chans = Load_IBW(path)
            if not os.path.exists(directory+'Processed\\'+filename[:-4]): os.mkdir(directory+'Processed\\'+filename[:-4])
            SaveChans(Chans,directory+'Processed\\'+filename[:-4]+'\\')
            del Chans
            print(filename+' Done')
            #break
            continue
        else:
            continue
            
    print('All Done')

In [16]:
directory = r'C:\Users\mgr69\Box Sync\Robertson-Tawfick\AFM\2018-09-12\\'
#directory = r'D:\AFM\Robertson\20181015\LFM\\'

In [17]:
Cypher_IBW(directory)

K_Pd_AMFM0000.ibw Done
K_Pd_AMFM0001.ibw Done
K_Pd_AMFM0002.ibw Done
K_Pd_AMFM0003.ibw Done
K_Pd_AMFM0004.ibw Done
K_Pd_AMFM0005.ibw Done
K_Pd_AMFM0006.ibw Done
K_Pd_AMFM0007.ibw Done
K_Pd_AMFM0008.ibw Done
K_Pd_AMFM0009.ibw Done
K_Pd_AMFM0010.ibw Done
K_Pd_AMFM0011.ibw Done
K_Pd_AMFM0012.ibw Done
K_Pd_AMFM0013.ibw Done
All Done
